In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 네이버 영화 리뷰 데이터로 검색
train = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train.head(2)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1


In [3]:
train.shape, test.shape

((150000, 3), (50000, 3))

# 1. 데이터 전처리

## - train data set

In [4]:
# 중복 여부 확인
train.document.nunique()

146182

In [5]:
# 중복 데이터 제거
train.drop_duplicates(subset=['document'],inplace=True)
train.shape

(146183, 3)

In [13]:
# null 데이터가 있는지 확인
train.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [7]:
# null 데이터 제거
train_df = train.dropna(how='any')
train_df.shape

(146182, 3)

In [9]:
# 긍정, 부정 레이블의 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

## -test data set

In [10]:
# 중복 여부 확인
test.document.nunique()

49157

In [11]:
# 중복 데이터 제거
test.drop_duplicates(subset=['document'],inplace=True)
test.shape

(49158, 3)

In [12]:
# null 데이터가 있는지 확인
test.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [15]:
# null 데이터 제거
test_df = test.dropna(how='any')
test_df.shape

(49157, 3)

In [16]:
# 긍정, 부정 레이블의 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

# 2. 텍스트 전처리

In [23]:
import warnings
warnings.filterwarnings(action='ignore')

## -Train data set

In [33]:
train_df['document'] = train_df.document.str.replace('^[ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head(2)

,id,document,label
0,9976970,더빙.. 진짜 짜증나네요 목소리,0
1,3819312,...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1


In [34]:
# '' 데이터는 Nan으로 변환한 후 제거
train_df['document'].replace('',np.nan, inplace=True)
train_df.document.isnull().sum()

75

In [35]:
train_df = train_df.dropna(how='any')
train_df.head(2)

,id,document,label
0,9976970,더빙.. 진짜 짜증나네요 목소리,0
1,3819312,...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1


## -Test data set

In [32]:
test_df['document'] = test_df.document.str.replace('^[ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df['document'].replace('',np.nan, inplace=True)
test_df = test_df.dropna(how='any')
test_df.shape

(49113, 3)

In [30]:
train_df.to_csv('naver_movie_train.tsv',sep='\t')
test_df.to_csv('naver_movie_test.tsv',sep='\t')

# 3. 한글처리

In [31]:
from konlpy.tag import Okt
okt = Okt()

In [36]:
stopwords = ['의','가','이','은','들','는','좀','잘','을','를','으로','걍','과','더','도','저','하다','ㅋㅋ','ㅠㅠ','ㅎㅎ']

In [37]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [38]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [39]:
# 형태소 분석 -> [] -> stopwords 제거 -> 문장으로 만들어주기
# X_train = [] 만든 후, join
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_train.append(temp_X)

In [41]:
X_test = []
for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_test.append(temp_X)

In [44]:
y_train = train_df.label.values
y_test = test_df.label.values

# 4. Feature변환, 모델 학습/예측/평가

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [46]:
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [47]:
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)

LogisticRegression()

In [48]:
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.8128601388634373

# 5. 실제 테스트

In [62]:
review1 = '아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~'
review2 = '메시지와 작위성의 불협화음!!!!'

In [64]:
review1 = review1.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
review1

'아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요'

In [63]:
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review1)
review1

'아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요'

In [65]:
morphs = okt.morphs(review1)
review = ' '.join([word for word in morphs if not word in stopwords])
review

'아름다운 음악 아름다운 풍 광 그렇지 못 한 현실 찡하네요'

In [56]:
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

1

In [60]:
review2 = review2.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
morphs = okt.morphs(review2)
review = ' '.join([word for word in morphs if not word in stopwords])
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

0

In [61]:
reviews = ['아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~','메시지와 작위성의 불협화음!!!']

In [66]:
review_list = []
for review in reviews:
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review)
    morphs = okt.morphs(review,stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    review_list.append(temp_X)

In [67]:
review_cv = cvect.transform(review_list)
pred = lr.predict(review_cv)
pred

array([1, 0], dtype=int64)

# 6. GridSearchCV로 최적 파라미터 찾기

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [73]:
pipeline=Pipeline([
    ('cvect', CountVectorizer()),
    ('lr', LogisticRegression())
])
params = {
    'cvect__ngram_range':[(1,1),(1,2)],
    'cvect__max_df':[0.9,0.99],
    'cvect__min_df':[1,3],
}

In [76]:
grid_pipe=GridSearchCV(
    pipeline, param_grid=params, cv=3, scoring='accuracy', n_jobs=1
)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_score_, grid_pipe.best_params_)

0.8270787742169813 {'cvect__max_df': 0.9, 'cvect__min_df': 1, 'cvect__ngram_range': (1, 2)}


In [78]:
pred = grid_pipe.best_estimator_.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'CounterVecorizer + LogisticRegression 정확도: {acc:.4f}')

CounterVecorizer + LogisticRegression 정확도: 0.8318
